In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import sympy as smp

from scipy.optimize import fsolve, leastsq, root
from scipy.integrate import odeint, solve_ivp
from Derivative_Equations import dvdr, dPdr, drhodr, dLdr
from Solving_ODE import Solve_ODE, derivatives, Crit_derivatives

In [2]:
h = 0.01
G = 1.33
aa = 0
#B_Max = 10**-50

In [15]:
def Crit(S, a):
    r, v, rho, Pressure, L = S
    Br = np.sqrt(4*np.pi*abs(Pressure)/300)
    #Br = B_Max
    Bphi = Br
    eq1, temp = dvdr([r, v, Pressure, rho, L, Br, Bphi, a])
    eq2, den = dPdr([r, v, Pressure, rho, L, Br, Bphi, a])
    eq3, temp = drhodr([r, v, Pressure, rho, L, Br, Bphi, a])
    eq4, temp = dLdr([r, v, Pressure, rho, L, Br, Bphi, a])
    # Use this when using fsolve
    # return [eq1, eq2, eq3, den]
    
    # Use this when using leastsq
    return [eq1, eq2, eq3, eq4, den]

r_ini_guess = 6
v_ini_guess = 0.3
rho_ini_guess = 10**-20
P_ini_guess = 10**-22
L_ini_guess = 3.2

ini_guess = np.array([r_ini_guess, v_ini_guess, rho_ini_guess, P_ini_guess, L_ini_guess])

In [16]:
r_crit, v_crit, rho_crit, P_crit, L_crit = fsolve(Crit, ini_guess, args=(aa), maxfev = 20000)
print(f"r_crit = {r_crit}")
print(f"v_crit = {v_crit}")
print(f"P_crit = {P_crit}")
print(f"rho_crit = {rho_crit}")
print(f"L_crit = {L_crit}")

Br_crit = np.sqrt(4*np.pi*abs(P_crit)/300)
#Br_crit = B_Max
Bphi_crit = Br_crit

S = [r_crit, v_crit, P_crit, rho_crit, L_crit, Br_crit, Bphi_crit, aa]
num1, den1 = dvdr(S)
num2, den2 = dPdr(S)
num3, den3 = drhodr(S)
num4, den4 = dLdr(S)

print(f"num1 = {num1}, den1 = {den1}")
print(f"num2 = {num2}, den2 = {den2}")
print(f"num3 = {num3}, den3 = {den3}")
print(f"num4 = {num4}, den4 = {den4}")
print(f"B_r = {Br_crit}")

r_crit = 76.25377093027898
v_crit = 0.07976246988391444
P_crit = 5.2731471662979455e-23
rho_crit = 9.669935313133717e-21
L_crit = 0.14989067696156314
num1 = 0.0008735460465523193, den1 = 2.2516409316739502e-05
num2 = -6.744953611169624e-25, den2 = 2.2516409316608588e-05
num3 = -8.979887488317372e-23, den3 = 2.2516409316529742e-05
num4 = 1.4345192706750796e-05, den4 = 2.2516409316642113e-05
B_r = 1.486206822706246e-12


In [5]:
def crit_der_down(S, L, a):
    dvdr_crit, dPdr_crit, drhodr_crit, dLdr_crit = S
    r = r_crit - h
    v = v_crit - h*dvdr_crit
    Pressure = P_crit - h*dPdr_crit
    rho = rho_crit - h*drhodr_crit
    L_calc = L_crit + h*dLdr_crit     
    Br = np.sqrt(4*np.pi*abs(Pressure)/300)
    #Br = B_Max
    Bphi = Br
    eq1 = dvdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[0]/dvdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[1] - dvdr_crit
    eq2 = dPdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[0]/dPdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[1] - dPdr_crit
    eq3 = drhodr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[0]/drhodr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[1] - drhodr_crit
    eq4 = dLdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[0]/dLdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[1] - dLdr_crit
    return [eq1, eq2, eq3, eq4]


def crit_der_up(S, L, a):
    dvdr_crit, dPdr_crit, drhodr_crit, dLdr_crit = S
    r = r_crit + h
    v = v_crit + h*dvdr_crit
    Pressure = P_crit + h*dPdr_crit
    rho = rho_crit + h*drhodr_crit
    L_calc = L + h*dLdr_crit 
    Br = np.sqrt(4*np.pi*abs(Pressure)/300)
    #Br = B_Max
    Bphi = Br
    eq1 = dvdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[0]/dvdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[1] - dvdr_crit
    eq2 = dPdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[0]/dPdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[1] - dPdr_crit
    eq3 = drhodr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[0]/drhodr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[1] - drhodr_crit
    eq4 = dLdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[0]/dLdr([r, v, Pressure, rho, L_calc, Br, Bphi, a])[1] - dLdr_crit
    return [eq1, eq2, eq3, eq4]

In [6]:
dvdr_ini_guess = -0.4
dPdr_ini_guess = -10
drhodr_ini_guess = -25
dLdr_ini_guess = 0.01

crit_der_ini_guess = [dvdr_ini_guess, dPdr_ini_guess, drhodr_ini_guess, dLdr_ini_guess]

In [7]:
v_crit_up, P_crit_up, rho_crit_up, L_crit_up = fsolve(crit_der_up, crit_der_ini_guess, args = (L_ini_guess, aa), maxfev = 2000)
print(f"v_crit_up = {v_crit_up}")
print(f"P_crit_up = {P_crit_up}")
print(f"rho_crit_up = {rho_crit_up}")
print(f"L_crit_up = {L_crit_up}")
crit_der_up([v_crit_up, P_crit_up, rho_crit_up, L_crit_up], L_ini_guess, aa)

v_crit_up = 0.0006078629572977064
P_crit_up = -5.304061244744573e-14
rho_crit_up = -8.885468936606225e-14
L_crit_up = -1.3859516019452731e-06


c:\Users\mythi\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The number of calls to function has reached maxfev = 2000.
  warnings.warn(msg, RuntimeWarning)


[4.063047641389428e-15,
 5.3040474908445406e-14,
 8.88545294668131e-14,
 -9.310374397982456e-18]

In [8]:
v_crit_down, P_crit_down, rho_crit_down, L_crit_down = fsolve(crit_der_down, crit_der_ini_guess, args = (L_ini_guess, aa))
print(f"v_crit_down = {v_crit_down}")
print(f"P_crit_down = {P_crit_down}")
print(f"rho_crit_down = {rho_crit_down}")
print(f"L_crit_down = {L_crit_down}")
crit_der_down([v_crit_down, P_crit_down, rho_crit_down, L_crit_down], L_ini_guess, aa)

v_crit_down = 0.0006182663022127283
P_crit_down = -3.5072184091845284e-12
rho_crit_down = -6.345761283333282e-12
L_crit_down = 2.9798380144172623e-06


c:\Users\mythi\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[3.616283911466833e-06,
 3.507229792563821e-12,
 6.3457763303506565e-12,
 -1.5827164605036465e-06]

In [9]:
def Solve_Ode_above(a, ini_guess):  
    # Setting up the critical values above and below rc
    S_up, S_down = Crit_derivatives(a, ini_guess)
    r_crit, v_crit, P_crit, rho_crit, dvdr_critical_up, dPdr_critical_up, drhodr_critical_up, dLdr_critical_up = S_up
    Crits_up = np.array([r_crit, v_crit, dvdr_critical_up, dPdr_critical_up, drhodr_critical_up, dLdr_critical_up])
    
    r_crit, v_crit, P_crit, rho_crit, dvdr_critical_down, dPdr_critical_down, drhodr_critical_down, dLdr_critical_down = S_down
    Crits_down = np.array([r_crit, v_crit, dvdr_critical_down, dPdr_critical_down, drhodr_critical_down, dLdr_critical_down])
    
    Crit_vals = Crits_up
    # Crit_vals = (Crits_down + Crits_up) / 2
    # Crit_vals = Crits_down
    
    #Solving the ODE below rc
    r_solve_below = np.arange(r_crit, 2.05, -0.001) 
    solution_below = odeint(derivatives, y0 = np.array([v_crit, P_crit, rho_crit, L_crit]), t = r_solve_below, tfirst = True, args = (L_crit, a, Crit_vals, B_Max))
    
    # Solving the ODE above rc
    r_solve_above = np.arange(r_crit, 8, 0.001) 
    solution_above = odeint(derivatives, y0 = np.array([v_crit, P_crit, rho_crit, L_crit]), t = r_solve_above, tfirst = True, args = (L_crit, a, Crit_vals, B_Max))
    
    #Reversing the np arrays for the solution below rc
    r_solve_below = np.flipud(r_solve_below)
    solution_below = np.flipud(solution_below) 
    
    #Combining the arrays for the solution
    r_solve = np.concatenate((r_solve_below, r_solve_above))
    solution = np.concatenate((solution_below, solution_above))
 
    v_r = np.array([])
    P_sol = np.array([])
    rho = np.array([])
    L_sol = np.array([])
    for i in range(len(r_solve)):
        v_r = np.concatenate((v_r, np.array([solution[i][0]])))
        P_sol = np.concatenate((P_sol, np.array([solution[i][1]])))
        rho = np.concatenate((rho, np.array([solution[i][2]])))
        L_sol = np.concatenate((L_sol, np.array([solution[i][3]])))
        
    #For Accretion
    if dvdr_critical_down < 0:
        str = "Accretion"
        
    else:
        str = "Wind"
    
    c_s = np.sqrt(G * abs(P_sol) / rho)
    
    #Plotting the final solution
    plt.plot(r_solve, c_s, label = "Speed of sound")
    plt.plot(r_solve, v_r, label = "Speed of gas")
    plt.grid()
    plt.xlabel("The distance from center of black hole (in r0 units)")
    plt.ylabel("Speed of sound/gas (assuming speed of light, c = 1)")
    plt.title("Plot of v_r and c_s vs r for " + str)
    plt.legend()
    plt.show()

    #Plotting the ratio of v_r and c_s
    plt.plot(r_solve, v_r/c_s)
    plt.grid()
    plt.xlabel("The distance from center of black hole (in r0 units)")
    plt.ylabel("Ratio of speed of gas to speed of sound")
    plt.title("Plot of v_r/c_s as a function of r in case of " + str)
    plt.show()

    #Plotting the Pressure
    plt.plot(r_solve, P_sol)
    plt.grid()
    plt.xlabel("The distance from center of black hole (in r0 units)")
    plt.ylabel("Pressure of the gas")
    plt.title("Plot of Pressure as a function of r in case of " + str)
    plt.show()

    #Plotting the density
    plt.plot(r_solve, rho)
    plt.grid()
    plt.xlabel("The distance from center of black hole (in r0 units)")
    plt.ylabel("Density of the gas")
    plt.title("Plot of Density as a function of r in case of " + str)
    plt.show()

    #Plotting the angular momentum
    plt.plot(r_solve, L_sol)
    plt.grid()
    plt.xlabel("The distance from center of black hole (in r0 units)")
    plt.ylabel("Angular momentum of the gas")
    plt.title("Plot of Angular momentum as a function of r in case of " + str)
    plt.show()

    final_solution = [v_r, P_sol, rho, L_sol]
    print(f"The critical radius is: {r_crit}")   
    print(f"The critical velocity derivative above is: {dvdr_critical_up}") 
    print(f"The critical velocity derivative below is: {dvdr_critical_down}") 
    print(solution[5][0])
    
    return final_solution, r_solve


In [10]:
Solve_Ode_above(aa, ini_guess)

r_crit = 4.793221019840798
v_crit = 0.34653537805762
P_crit = 1.5549206095444343e-22
rho_crit = 1.5106360767074732e-21
L_crit = 2.289484626837431
num1 = -0.0009915014811023894, den1 = 8.982404574124733e-13
num2 = 4.865999700422941e-25, den2 = 8.983924715266767e-13
num3 = -2.251145419296683e-25, den3 = 8.983676123633012e-13
num4 = 0.0009915013651814217, den4 = 8.982377034675785e-13
Plugging in the derivatives above rc: [-7.271823193627514e-05, 6.47795236772767e-05, 0.00011525888840889383, -4.977016491864437e-05]
Plugging in the derivatives below rc: [-0.0017316927040773589, 6.477952319847567e-05, 0.00011525889154229885, -0.0010612819835155024]


NameError: name 'B_Max' is not defined